In [80]:
import pandas as pd
from pyalex import Works, Authors, Sources, Institutions, Concepts, Publishers, Funders
from tqdm import tqdm
from itertools import chain
import json
from requests.exceptions import ConnectionError


In [3]:
df = pd.read_csv("data/works-2023-11-10T17-54-33.csv")

/var/folders/4f/ddlj81h90_n0_h5wwvjbd2b40000gn/T/ipykernel_82409/898392114.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/works-2023-11-10T17-54-33.csv")


In [4]:
df.columns

Index(['id', 'display_name', 'publication_date', 'relevance_score',
       'primary_location_id', 'primary_location_display_name',
       'primary_location_host_organization', 'primary_location_issns',
       'primary_location_issn_l', 'primary_location_type',
       'primary_location_landing_page_url', 'primary_location_pdf_url',
       'primary_location_is_oa', 'primary_location_version',
       'primary_location_license', 'author_ids', 'author_names',
       'author_orcids', 'author_institution_ids', 'author_institution_names',
       'is_oa', 'oa_status', 'oa_url', 'cited_by_count', 'doi', 'mag', 'pmid',
       'pmcid', 'publication_year', 'cited_by_api_url', 'type', 'is_paratext',
       'is_retracted', 'biblio_issue', 'biblio_first_page', 'biblio_volume',
       'biblio_last_page', 'referenced_works', 'related_works', 'concept_ids'],
      dtype='object')

In [9]:
df.iloc[0]

id                                                     https://openalex.org/W4205869927
display_name                                           Medieval Canon Law: Introduction
publication_date                                                             2022-01-27
relevance_score                                                               206.85774
primary_location_id                                    https://openalex.org/S4306462995
primary_location_display_name                         Cambridge University Press eBooks
primary_location_host_organization                                                  NaN
primary_location_issns                                                              NaN
primary_location_issn_l                                                             NaN
primary_location_type                                                    ebook platform
primary_location_landing_page_url             https://doi.org/10.1017/9781139177221.001
primary_location_pdf_url        

In [10]:
df.iloc[0].id

'https://openalex.org/W4205869927'

In [29]:
final_data = []
# Iterate over DataFrame rows
for index, row in tqdm(df.iterrows(), total=len(df)):
    try:
        work = Works()[row.id]
        if "abstract_inverted_index" in work:
            abstract = reconstruct_abstract(work["abstract_inverted_index"])
            work["abstract"] = abstract
        final_data.append(work)
    except KeyError as e:
        print(f"Key error: {e} in row index {index}")
    except Exception as e:
        print(f"An error occurred: {e} in row index {index}")

  0%|          | 7/24502 [01:05<64:03:52,  9.42s/it] 


KeyboardInterrupt: 

In [20]:
work = Works()[df.iloc[0].id]

In [25]:
def reconstruct_abstract(inverted_index):
    # Create a list of tuples, each containing a word and its first occurrence index
    word_positions = [(word, positions[0]) for word, positions in inverted_index.items()]

    # Sort the tuples based on the index
    sorted_words = sorted(word_positions, key=lambda x: x[1])

    # Extract the sorted words
    sorted_words_only = [word for word, position in sorted_words]

    # Join the words into a single string
    abstract = ' '.join(sorted_words_only)

    return abstract

In [26]:
abstract = reconstruct_abstract(work["abstract_inverted_index"])

In [31]:
ids = df.id.tolist()[:10]

In [37]:
works = Works()["|".join(ids)]

In [38]:
len(works)

42

In [39]:
ids

['https://openalex.org/W4205869927',
 'https://openalex.org/W4297854822',
 'https://openalex.org/W3093626553',
 'https://openalex.org/W4304591020',
 'https://openalex.org/W4293243082',
 'https://openalex.org/W4282599125',
 'https://openalex.org/W4200036874',
 'https://openalex.org/W3212247621',
 'https://openalex.org/W4205234211',
 'https://openalex.org/W4206701281']

In [67]:
query = Works().search("medieval").filter(publication_year=1960)

In [68]:
query.count()

2476

In [81]:
for year in range(1970, 2023):
    success = False

    while not success:
        try:
            query = Works().search("medieval").filter(publication_year=year)
            print(year, query.count())

            all_records = []  # Initialize or clear the list for each year
            for record in chain(*query.paginate(per_page=200, n_max=None)):
                all_records.append(record)

            with open(f'./data/{year}.json', 'w') as file:
                json.dump(all_records, file)

            success = True  # Mark this year as successfully processed

        except ConnectionError as e:
            print(f"ConnectionError encountered for year {year}. Retrying...")   

1970 4268
1971 3515
1972 3822
1973 4147
1974 4063
1975 4377
1976 4358
ConnectionError encountered for year 1976. Retrying...
1976 4358
ConnectionError encountered for year 1976. Retrying...
1976 4358
ConnectionError encountered for year 1976. Retrying...
1976 4358
1977 4853
ConnectionError encountered for year 1977. Retrying...
1977 4853
1978 4845
1979 4756
ConnectionError encountered for year 1979. Retrying...
1979 4756
ConnectionError encountered for year 1979. Retrying...
1979 4756
1980 5010
1981 5085
ConnectionError encountered for year 1981. Retrying...
1981 5085
ConnectionError encountered for year 1981. Retrying...
1981 5085
ConnectionError encountered for year 1981. Retrying...
1981 5085
1982 5361
ConnectionError encountered for year 1982. Retrying...
1982 5361
ConnectionError encountered for year 1982. Retrying...
1982 5361
1983 5529
1984 5810
1985 5954
1986 6397
1987 6906
ConnectionError encountered for year 1987. Retrying...
1987 6906
ConnectionError encountered for year 198

In [70]:
len(all_records)

200

In [ ]:
df = pd.DataFrame(all_records)
df.to_csv("data/1960.csv")

In [62]:
all_records[0]["abstract_inverted_index"]

{'This': [0],
 'is': [1, 56, 129],
 'the': [2, 14, 35, 43, 97, 108, 121, 141, 145, 171, 195, 199, 211, 218],
 'first': [3],
 'monograph-length': [4],
 'study': [5],
 'in': [6, 29, 76, 126, 174, 183],
 'English': [7],
 'of': [8, 13, 19, 27, 32, 103, 137, 160, 189, 201],
 'Kamo': [9],
 'no': [10],
 'Chōmei,': [11, 202],
 'one': [12],
 'most': [15],
 'important': [16, 131],
 'literary': [17, 81],
 'figures': [18],
 'medieval': [20, 157, 184],
 'Japan.': [21, 185],
 'Drawing': [22],
 'upon': [23, 114],
 'a': [24, 30, 86, 112, 134, 176, 207],
 'wide': [25],
 'range': [26],
 'writings': [28],
 'variety': [31],
 'genres': [33],
 'from': [34],
 'Heian': [36],
 'and': [37, 49, 61, 82, 124, 147, 210],
 'Kamakura': [38],
 'periods,': [39],
 'Pandey': [40],
 'focuses': [41],
 'on': [42, 170],
 'terms': [44, 72],
 'kyōgen': [45],
 'kigo': [46],
 '(wild': [47],
 'words': [48],
 'fancy': [50],
 'phrases),': [51],
 'shoji': [52],
 'soku': [53],
 'nehan': [54],
 '(samsara': [55],
 'nirvana),': [57],
 '